# Experiment - LM Word Accuracy

In [1]:
NAME = '09-03_experiment_lm_accuracy'
PROJECT = 'conference-calls-sentiment'
PYTHON_VERSION = '3.7.0'

### Imports

In [2]:
import os
import re
import pickle
import numpy as np
import pandas as pd

### Settings

In [3]:
workdir = re.sub("(?<={})[\w\W]*".format(PROJECT), "", os.getcwd())
os.chdir(workdir)

---
# Main Code 
## Read Phrases
### Financial Phrasebank

In [4]:
def read_labeled_sentences(level):
    with open(os.path.join('0_data', 'financial_phrasebank', f'Sentences_{level}Agree.txt')) as f:
        labeled_sentences = f.readlines()
    sentences, labels = [], []
    for i in labeled_sentences:
        sentence, label = i.split(('@'))
        sentences.append(sentence)
        labels.append(label.strip())
    return pd.DataFrame({'sentence': sentences, 'label': labels})

In [5]:
fpb = read_labeled_sentences('50')
len(fpb)

4846

### FiQa

In [6]:
NEUTRAL_THRESHOLD = 0.2

In [7]:
REPLACE_BY_SPACE = re.compile(r'[/(){}\[\]\|@,;-]')
BAD_SYMBOLS = re.compile(r'[^0-9a-z +_]')
HYPERLINKS = re.compile(r'(http)([^\s]+)')

def clean_text(transcript):
    transcript = transcript.lower()
    transcript = HYPERLINKS.sub('', transcript)
    transcript = REPLACE_BY_SPACE.sub(' ', transcript)
    transcript = BAD_SYMBOLS.sub('', transcript)
    transcript = re.sub(r'\s+', ' ', transcript)
    return transcript.strip()

In [8]:
headlines = pd.read_json(os.path.join('0_data', 'fiqa', 'task1_headline_ABSA_train.json')).transpose()

In [9]:
headlines['sentence'] = headlines['sentence'].apply(clean_text)

headlines['sentiment_score'] = (headlines['info']
                                .apply(lambda x: x[0]['sentiment_score'])
                                .astype(np.float))

headlines.loc[np.abs(headlines['sentiment_score']) <= NEUTRAL_THRESHOLD, 'label'] = 'neutral'
headlines.loc[headlines['sentiment_score'] > NEUTRAL_THRESHOLD, 'label'] = 'positive'
headlines.loc[headlines['sentiment_score'] < -NEUTRAL_THRESHOLD, 'label'] = 'negative'

headlines.drop(columns=['info'], inplace=True)

In [10]:
posts = pd.read_json(os.path.join('0_data', 'fiqa', 'task1_post_ABSA_train.json')).transpose()

In [11]:
posts['sentence'] = posts['sentence'].apply(clean_text)

posts['sentiment_score'] = (posts['info']
                            .apply(lambda x: x[0]['sentiment_score'])
                            .astype(np.float))

posts.loc[np.abs(posts['sentiment_score']) <= NEUTRAL_THRESHOLD, 'label'] = 'neutral'
posts.loc[posts['sentiment_score'] > NEUTRAL_THRESHOLD, 'label'] = 'positive'
posts.loc[posts['sentiment_score'] < -NEUTRAL_THRESHOLD, 'label'] = 'negative'

posts.drop(columns=['info'], inplace=True)

### Combine Phrases

In [12]:
phrases = pd.concat([fpb, headlines, posts])
len(phrases)

5957

## Sentiment

### Loughran & McDonald (2011)

In [13]:
with open(os.path.join('2_pipeline', '03-01_model_dictionaries', 'out', 'lm_positive.pickle'), 'rb') as f:
    lm_positive = pickle.load(f)

with open(os.path.join('2_pipeline', '03-01_model_dictionaries', 'out', 'lm_negative.pickle'), 'rb') as f:
    lm_negative = pickle.load(f)

print(f"# Positive: {len(lm_positive)}\n# Negative: {len(lm_negative)}")

# Positive: 354
# Negative: 2355


In [14]:
def count_from_list(sentence, word_list):
    '''Returns the count of words in word_list in conference calls'''
    sentence_words = sentence.lower().split()
    num = 0
    for word in word_list:
        num += sentence_words.count(word)
    return num

def sentiment_words(sentence, word_list):
    '''Returns the count of words in word_list in conference calls'''
    sentence_words = sentence.lower().split()
    words = []
    for word in sentence_words:
        if word in word_list:
            words.append(word)
    return words

def get_sentiment(row):
    '''Returns the sentiment classification'''
    if row['num_positive'] == row['num_negative']:
        return 'neutral'
    elif row['num_positive'] > row['num_negative']:
        return 'positive'
    else:
        return 'negative'

In [15]:
phrases['num_negative'] = phrases['sentence'].apply(count_from_list, word_list=lm_negative)
phrases['num_positive'] = phrases['sentence'].apply(count_from_list, word_list=lm_positive)

phrases['words'] = phrases['sentence'].apply(sentiment_words, word_list=lm_positive + lm_negative)

phrases['lm_dictionary'] = phrases.apply(get_sentiment, axis=1)

In [16]:
sentiment_words = (phrases
                   .assign(word=lambda x: x['words'])
                   .explode('word')
                   .drop_duplicates(['sentence', 'word']))
sentiment_words

,sentence,label,sentiment_score,num_negative,num_positive,words,lm_dictionary,word
0,"According to Gran , the company has no plans t...",neutral,NaN,0,0,[],neutral,NaN
1,Technopolis plans to develop in stages an area...,neutral,NaN,0,0,[],neutral,NaN
2,The international electronic industry company ...,negative,NaN,2,0,"[contrary, layoffs]",negative,contrary
2,The international electronic industry company ...,negative,NaN,2,0,"[contrary, layoffs]",negative,layoffs
3,With the new production plant the company woul...,positive,NaN,0,2,"[improve, profitability]",positive,improve
...,...,...,...,...,...,...,...,...
19130,facebook fb received a buy rating from wells f...,positive,0.456,0,0,[],neutral,NaN
19149,tsla wish had my puts back but see if we can f...,negative,-0.485,0,0,[],neutral,NaN
19161,citrix systems inc ctxs position increased by ...,positive,0.529,0,0,[],neutral,NaN
19163,notable gainers among liquid option names this...,positive,0.513,0,0,[],neutral,NaN


In [25]:
word_overview = (sentiment_words
                 .loc[sentiment_words['words'].str.len() == 1]
                 ['word'].value_counts()
                 .reset_index()
                 .rename(columns={'index': 'word', 'word': 'frequency'})
                 .assign(lm_sentiment=lambda x: x['word'].apply(lambda x: 'Positive' if x in lm_positive else 'Negative')))

In [26]:
word_overview

,word,frequency,lm_sentiment
0,loss,132,Negative
1,positive,35,Positive
2,strong,34,Positive
3,leading,34,Positive
4,good,33,Positive
...,...,...,...
342,rebound,1,Positive
343,disgraceful,1,Negative
344,turmoil,1,Negative
345,wrong,1,Negative


In [31]:
def get_actual_sentiment(word):
    return (sentiment_words
            .loc[sentiment_words['words'].str.len() == 1]
            .loc[lambda x: x['word'] == word]
            .drop_duplicates('sentence')
            ['label'].value_counts())

In [33]:
word_overview[['Negative', 'Positive', 'Neutral']] = word_overview['word'].apply(lambda x: get_actual_sentiment(x)).fillna(0).astype('int')

In [34]:
word_overview['num_correct'] = word_overview.apply(lambda x: x[x['lm_sentiment']], axis=1)
word_overview['Accuracy'] = word_overview['num_correct'] / word_overview['frequency']

## Tables

In [40]:
(word_overview
.query("frequency >= 5 & lm_sentiment == 'Positive'")
.sort_values('Accuracy', ascending=False)
.head(8)
.set_index('word')
.rename(columns={'frequency': 'Frequency',
                 'lm_sentiment': 'LM Dictionary'})
.assign(Accuracy=lambda x: x['Accuracy'].apply(lambda s: f'{s:.2%}'))
.filter(['Frequency', 'Positive',
         'Neutral', 'Negative', 'Accuracy']))

,Frequency,Positive,Neutral,Negative,Accuracy
word,,,,,
improved,17,17,0,0,100.00%
boosted,7,7,0,0,100.00%
outperform,8,8,0,0,100.00%
positive,35,33,1,1,94.29%
good,33,28,4,1,84.85%
happy,6,5,1,0,83.33%
efficient,6,5,1,0,83.33%
pleased,10,8,2,0,80.00%


In [44]:
(word_overview
.query("frequency >= 5 & lm_sentiment == 'Positive'")
.sort_values('Accuracy', ascending=False)
.tail(8)
.set_index('word')
.rename(columns={'frequency': 'Frequency',
                 'lm_sentiment': 'LM Dictionary'})
.assign(Accuracy=lambda x: x['Accuracy'].apply(lambda s: f'{s:.2%}'))
.filter(['Frequency', 'Positive',
         'Neutral', 'Negative', 'Accuracy']))

,Frequency,Positive,Neutral,Negative,Accuracy
word,,,,,
opportunities,5,2,3,0,40.00%
innovative,6,2,4,0,33.33%
popular,6,2,4,0,33.33%
enable,14,3,11,0,21.43%
enabling,5,1,4,0,20.00%
enables,13,2,11,0,15.38%
effective,8,1,6,1,12.50%
invention,5,0,5,0,0.00%


In [173]:
(word_overview
.query("frequency >= 5 & lm_sentiment == 'Negative'")
.sort_values('Accuracy', ascending=False)
.head(8)
.set_index('word')
.rename(columns={'frequency': 'Frequency',
                 'lm_sentiment': 'LM Dictionary'})
.assign(Accuracy=lambda x: x['Accuracy'].apply(lambda s: f'{s:.2%}'))
.filter(['Frequency', 'Positive',
         'Neutral', 'Negative', 'Accuracy']))

,Frequency,Positive,Neutral,Negative,Accuracy
word,,,,,
warning,12,0,0,12,100.00%
dropped,15,1,1,13,86.67%
weak,17,2,2,13,76.47%
cut,30,3,6,21,70.00%
recall,13,2,2,9,69.23%
declined,16,1,4,11,68.75%
negative,19,6,0,13,68.42%
against,14,3,4,7,50.00%


In [174]:
(word_overview
.query("frequency >= 5 & lm_sentiment == 'Negative'")
.sort_values('Accuracy', ascending=False)
.tail(8)
.set_index('word')
.rename(columns={'frequency': 'Frequency',
                 'lm_sentiment': 'LM Dictionary'})
.assign(Accuracy=lambda x: x['Accuracy'].apply(lambda s: f'{s:.2%}'))
.filter(['Frequency', 'Positive',
         'Neutral', 'Negative', 'Accuracy']))

,Frequency,Positive,Neutral,Negative,Accuracy
word,,,,,
breaking,12,8,2,2,16.67%
break,21,12,6,3,14.29%
closing,11,5,5,1,9.09%
bridge,18,9,9,0,0.00%
disclosed,24,0,24,0,0.00%
disclose,12,0,12,0,0.00%
undisclosed,10,2,8,0,0.00%
divestment,10,3,7,0,0.00%
